### Unpack ETL files

In [2]:
import os
from pathlib import Path

from openai.types.beta.threads import image_file

project_root = Path().resolve()
etl_dir = project_root / "data" / "ETL8G"
unpack_script = project_root / "data" / "unpack_etlcdb" / "unpack_etlcdb" / "unpack.py"

files = os.listdir(etl_dir)

for file in files:
    if file != "ETL8INFO":
        input_file = etl_dir / file
        cmd = f'python {unpack_script} {input_file}'
        print("Running:", cmd)
        # os.system(cmd)



Running: python C:\Users\alicj\PycharmProjects\KanjiRecognitionModel\data\unpack_etlcdb\unpack_etlcdb\unpack.py C:\Users\alicj\PycharmProjects\KanjiRecognitionModel\data\ETL8G\ETL8G_01
Running: python C:\Users\alicj\PycharmProjects\KanjiRecognitionModel\data\unpack_etlcdb\unpack_etlcdb\unpack.py C:\Users\alicj\PycharmProjects\KanjiRecognitionModel\data\ETL8G\ETL8G_01_unpack
Running: python C:\Users\alicj\PycharmProjects\KanjiRecognitionModel\data\unpack_etlcdb\unpack_etlcdb\unpack.py C:\Users\alicj\PycharmProjects\KanjiRecognitionModel\data\ETL8G\ETL8G_02
Running: python C:\Users\alicj\PycharmProjects\KanjiRecognitionModel\data\unpack_etlcdb\unpack_etlcdb\unpack.py C:\Users\alicj\PycharmProjects\KanjiRecognitionModel\data\ETL8G\ETL8G_02_unpack
Running: python C:\Users\alicj\PycharmProjects\KanjiRecognitionModel\data\unpack_etlcdb\unpack_etlcdb\unpack.py C:\Users\alicj\PycharmProjects\KanjiRecognitionModel\data\ETL8G\ETL8G_03
Running: python C:\Users\alicj\PycharmProjects\KanjiRecogniti

### Saving labels to list

In [59]:
import pandas as pd

path_to_labels = Path().resolve()/ "data" / "ETL8G"/ "ETL8G_01_unpack"/"meta.csv"

labels_df = pd.read_csv(path_to_labels)

labels = labels_df["char"]
labels = labels.tolist()


In [60]:
all_labels = labels.copy()
labels_copy = all_labels.copy()

for i in range(31):
    labels_copy_2 = labels_copy.copy()
    all_labels.extend(labels_copy_2)

all_labels.extend(labels_copy[:956])

In [8]:
# from sklearn.preprocessing import LabelEncoder
#
# le = LabelEncoder()
# labels_int = le.fit_transform(labels)
#
# print(labels_int[:10])        # now numbers like [0, 1, 0, 2, ...]
# print(le.classes_[:10])       # the mapping back to original labels


[  0 415 301 281 919 162 611 118 588 111]
['あ' 'い' 'う' 'え' 'お' 'か' 'が' 'き' 'ぎ' 'く']


In [61]:
# go through each folder
path_label_list = []
etl_dir = Path().resolve() / "data" / "ETL8G"
for folder in os.listdir(etl_dir):
    if "unpack" in folder:
        folder_path = os.path.join(etl_dir, folder)

        # go through each png in folder
        for fname in os.listdir(folder_path):
            if fname.endswith(".png"):
                fpath = os.path.join(folder_path, fname)
                # take filename without extension
                idx = int(os.path.splitext(fname)[0])
                # compute label
                label = idx % 956
                path_label_list.append((fpath, label))

print(path_label_list[956])


('C:\\Users\\alicj\\PycharmProjects\\KanjiRecognitionModel\\data\\ETL8G\\ETL8G_01_unpack\\00956.png', 0)


In [62]:
print(len(path_label_list))

153916


In [68]:
print(labels[1])
print(path_label_list[1912])

愛
('C:\\Users\\alicj\\PycharmProjects\\KanjiRecognitionModel\\data\\ETL8G\\ETL8G_01_unpack\\01912.png', 0)


### Saving images

In [72]:
image_paths = []
img_dir_path = Path().resolve()/ "data" / "ETL8G"

files = os.listdir(img_dir_path)
for file in files:
    if "unpack" in file:
        unpack_dir = img_dir_path / file
        img_files = os.listdir(unpack_dir)
        for img_file in img_files:
            if img_file != 'meta.csv':
                img_path = unpack_dir / img_file
                image_paths.append(img_path)

# in the last dir there are only 955 images



### Create a CNN

In [73]:
print(len(image_paths))
print(len(labels_int))

153916
153916


In [7]:
print(len(labels))

153916


### Dataset class

In [125]:
from torch.utils.data import Dataset
from PIL import Image
import torch
import torchvision.transforms as T

class ImageDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        path = item[0]
        img = Image.open(path)
        img=img.convert("L")
        img = T.ToTensor()(img)
        label =item[1]
        # img.show()
        return img, torch.tensor(label, dtype=torch.long)



In [127]:
a = ImageDataset(path_label_list)
img, lab = a[0]
# print(img.size)
print(lab)
print(type(img), img.shape, lab)

tensor(0)
<class 'torch.Tensor'> torch.Size([1, 127, 128]) tensor(0)


### Splitting the data into train, val, test sets

In [128]:
from sklearn.model_selection import train_test_split

# first split train vs temp (val+test)
train_data, temp_data = train_test_split(path_label_list, test_size=0.3, random_state=42, stratify=[d[1] for d in path_label_list]) # used tratify to preserve the same class proportions

# then split temp into val and test
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42, stratify=[d[1] for d in temp_data])


In [129]:
train_dataset = ImageDataset(train_data)
val_dataset   = ImageDataset(val_data)
test_dataset  = ImageDataset(test_data)


### Data Loader

In [130]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=32)
test_loader  = DataLoader(test_dataset, batch_size=32)


### Test CNN

In [134]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        # 1 input channel (grayscale), 8 filters
        self.conv1 = nn.Conv2d(1, 8, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)   # reduces size by half
        # Flattened size after pool (assuming 64x64 input): 8 * 32 * 32
        self.fc1 = nn.Linear(32256, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # conv → relu → pool
        x = x.view(x.size(0), -1)             # flatten
        x = self.fc1(x)                       # fully connected
        return x


In [135]:
# Example: suppose you have 100 kanji classes
model = SimpleCNN(num_classes=956)

# One fake batch: 32 grayscale images, 64x64 each
x = torch.randn(32, 1, 64, 64)
out = model(x)

print(out.shape)  # torch.Size([32, 100]) → 32 predictions, 100 classes


RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x8192 and 32256x956)

In [138]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(3):  # 3 training epochs
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()         # reset gradients
        outputs = model(images)       # forward pass
        loss = criterion(outputs, labels)
        loss.backward()               # backward pass
        optimizer.step()              # update weights
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")


KeyboardInterrupt: 